In [1]:
import sys
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score

### Custom files ###
dir_gp_constr = 'C:\\Users\\tln229\\Downloads\\Python Tests\\Constrained GP_2\\GP_2\\gp_constr\\'
sys.path.append(dir_gp_constr)

# Import
from GPConstr.model import GPmodel, Constraint
from GPConstr.kern import kernel_Matern52

Loading constrained GP module from C:\Users\tln229\Downloads\Python\1. Building\Finals\1. Zone model


Unable to determine R home: [WinError 2] The system cannot find the file specified


Loading R wrapper...


R[write to console]: Loading required package: mvtnorm

R[write to console]: Loading required package: Matrix

R[write to console]: Loading required package: stats4

R[write to console]: Loading required package: gmm

R[write to console]: Loading required package: sandwich

R[write to console]: 
Attaching package: 'TruncatedNormal'


R[write to console]: The following objects are masked from 'package:tmvtnorm':

    dtmvnorm, dtmvt, ptmvnorm, ptmvt, rtmvnorm, rtmvt


R[write to console]: The following objects are masked from 'package:mvtnorm':

    pmvnorm, pmvt




Running R from rpy2: R version 4.2.2 (2022-10-31 ucrt)


In [2]:
def get_diagnostics_df(x_test, y_test, mean, var):
    """ Put test data in a dataframe"""
    df = pd.DataFrame(x_test)
    df.columns = ['x' + str(i+1) for i in range(x_test.shape[1])]
    df['y_true'] = y_test
    df['y_mean'] = mean
    df['y_var'] = var
    return df

In [3]:
def constant_function(val):
    def fun(x):
        return np.array([val]*x.shape[0])
    return fun

In [4]:
# ---------- DATA ----------
def create_data(n_train, n_Xv):
    df = pd.read_csv('C:/Users/tln229/Downloads/Python/1. Building/data/HVAC_B90_106_exp_30m_20221130.csv')
    n_end = 3000

    # TRAIN SET
    Tz_k   = np.array(df['room_temp']).reshape(-1,1)[0: n_train]
    To_k   = np.array(df['thermostat_outside_temp']).reshape(-1,1)[0: n_train]
    Tz_k1  = np.array(df['room_temp'])[1: n_train+1]
    Tsa_k1 = np.array(df['supply_discharge_temp']).reshape(-1,1)[1: n_train+1]
    msa_k1 = np.array(df['airflow_current']).reshape(-1,1)[1: n_train+1]
    z      = ((msa_k1-400)/600)*(Tsa_k1-Tz_k)

    x_train = np.concatenate((Tz_k, Tsa_k1, z, To_k), axis=1)
    y_train = Tz_k1

    # Xv
    x1v = np.linspace(69, 75, n_Xv).reshape(-1, 1)
    x2v = np.linspace(56, 96, n_Xv).reshape(-1, 1)
    x3v = np.linspace(-3, 15, n_Xv).reshape(-1, 1)
    x4v = np.linspace(35, 51, n_Xv).reshape(-1, 1)
    Xv  = np.concatenate((x1v, x2v, x3v, x4v), axis = 1)

    # TEST SET
    Tz_k_t   = np.array(df['room_temp']).reshape(-1,1)[n_train: n_end]
    Tz_o_t   = np.array(df['thermostat_outside_temp']).reshape(-1,1)[n_train: n_end]
    Tz_k1_t  = np.array(df['room_temp'])[n_train+1: n_end+1]
    Tsa_k1_t = np.array(df['supply_discharge_temp']).reshape(-1,1)[n_train+1: n_end+1]
    msa_k1_t = np.array(df['airflow_current']).reshape(-1,1)[n_train+1: n_end+1]
    z_t      = ((msa_k1_t-400)/600)*(Tsa_k1_t-Tz_k_t)

    x_test = np.concatenate((Tz_k_t, Tsa_k1_t, z_t, Tz_o_t), axis=1)
    y_test = Tz_k1_t

    return x_train, y_train, Xv, x_test, y_test

In [5]:
df_result = pd.DataFrame({'n_train':[], 'n_Xv':[], 'R2':[]})

for n_train in ([32, 64, 128]):
    for n_Xv in ([5, 10, 20]):
        # DATA
        x_train, y_train, Xv, x_test, y_test = create_data(n_train=n_train, n_Xv=n_Xv)

        # MODEL
        ker = kernel_Matern52(variance = 1, lengthscale = [1]*x_train.shape[1])
        model = GPmodel(kernel = ker, likelihood = 1, mean = 0)

        model.X_training = x_train
        model.Y_training = y_train

        model.optimize(include_constraint = False, fix_likelihood = False)

        # CONSTRAINTS
        model.constr_deriv = [
            Constraint(LB = constant_function(float('-Inf')), UB = constant_function(float('Inf'))),
            Constraint(LB = constant_function(0), UB = constant_function(float('Inf'))),
            Constraint(LB = constant_function(float('-Inf')), UB = constant_function(float('Inf'))),
            Constraint(LB = constant_function(float('-Inf')), UB = constant_function(float('Inf')))
        ]

        model.constr_deriv[0].Xv = Xv
        model.constr_deriv[1].Xv = Xv
        model.constr_deriv[2].Xv = Xv
        model.constr_deriv[3].Xv = Xv

        model.constr_likelihood = 1E-6

        # PREDICT
        mean, _, _, _, _, _ = model.calc_posterior_constrained(x_test, num_samples = 500, save_samples = 50)
        mean = np.array(mean).flatten()
        df_result.loc[len(df_result)] = [n_train, n_Xv, r2_score(y_test, mean)]

..Running optimization for unconstrained GP ... DONE - Total time: 0.128 seconds
..Running calculation of K_w ... SKIP - (cached)
..Running calculation of Cholesky factor for K_w ... SKIP - (cached)
..Running preparation step 1 - dependence on (XS, X) ... DONE - time: 0.305 seconds
..Running preparation step 2 - dependence on (XV, X) ... DONE - time: 0.003 seconds
..Running preparation step 3 - dependence on (XS, XV, X) ... DONE - time: 0.055 seconds
..sampling 500 times from truncated constraint distribution C~|C, Y DONE - time: 0.030 seconds
..sampling 500 times from constrained GP f*|C, Y DONE - time: 19.173 seconds
..computing statistics from samples DONE - time: 0.049 seconds
 DONE - Total time: 19.617 seconds
..Running optimization for unconstrained GP ... DONE - Total time: 0.128 seconds
..Running calculation of K_w ... SKIP - (cached)
..Running calculation of Cholesky factor for K_w ... SKIP - (cached)
..Running preparation step 1 - dependence on (XS, X) ... DONE - time: 0.292 

In [6]:
print(df_result)

   n_train  n_Xv        R2
0     32.0   5.0 -7.811431
1     32.0  10.0 -8.097641
2     32.0  20.0 -7.810607
3     64.0   5.0  0.655091
4     64.0  10.0  0.637723
5     64.0  20.0  0.646984
6    128.0   5.0  0.991243
7    128.0  10.0  0.990883
8    128.0  20.0  0.990989
